<a href="https://colab.research.google.com/github/ChiragBangera/MLspark/blob/main/readWriteHW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Section 1, 2 HW

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=e0bbb92d0fd7dfcc1af27c15ef2a66f499b65d5575e1760b33ff4a812c798b36
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as f

In [ ]:
spark = SparkSession.builder.appName('ReadwriteHW').getOrCreate()

In [ ]:
pga_schema = StructType([
    StructField('Player Name', StringType(),True),
    StructField('Season',StringType(),True),
    StructField('Statistic',StringType(),True),
    StructField('Variable',StringType(),True),
    StructField('Value',IntegerType(),True)
])

In [ ]:
path = '/content/drive/MyDrive/spark/datasets/'
pga = spark.read.csv(path+'pga_tour_historical.csv',schema=pga_schema, header=True)
pga.show()

+---------------+------+----------------+--------------------+-----+
|    Player Name|Season|       Statistic|            Variable|Value|
+---------------+------+----------------+--------------------+-----+
|Robert Garrigus|  2010|Driving Distance|Driving Distance ...|   71|
|   Bubba Watson|  2010|Driving Distance|Driving Distance ...|   77|
| Dustin Johnson|  2010|Driving Distance|Driving Distance ...|   83|
|Brett Wetterich|  2010|Driving Distance|Driving Distance ...|   54|
|    J.B. Holmes|  2010|Driving Distance|Driving Distance ...|  100|
|      John Daly|  2010|Driving Distance|Driving Distance ...|   63|
|  Graham DeLaet|  2010|Driving Distance|Driving Distance ...|   88|
|  Angel Cabrera|  2010|Driving Distance|Driving Distance ...|   64|
| Charles Warren|  2010|Driving Distance|Driving Distance ...|   64|
|    D.J. Trahan|  2010|Driving Distance|Driving Distance ...|   92|
|  Martin Flores|  2010|Driving Distance|Driving Distance ...|   75|
|   Rory McIlroy|  2010|Driving Di

In [ ]:
pga.show(5)

+---------------+------+----------------+--------------------+-----+
|    Player Name|Season|       Statistic|            Variable|Value|
+---------------+------+----------------+--------------------+-----+
|Robert Garrigus|  2010|Driving Distance|Driving Distance ...|   71|
|   Bubba Watson|  2010|Driving Distance|Driving Distance ...|   77|
| Dustin Johnson|  2010|Driving Distance|Driving Distance ...|   83|
|Brett Wetterich|  2010|Driving Distance|Driving Distance ...|   54|
|    J.B. Holmes|  2010|Driving Distance|Driving Distance ...|  100|
+---------------+------+----------------+--------------------+-----+
only showing top 5 rows



In [ ]:
pga.toPandas()

,Player Name,Season,Statistic,Variable,Value
0,Robert Garrigus,2010,Driving Distance,Driving Distance - (ROUNDS),71
1,Bubba Watson,2010,Driving Distance,Driving Distance - (ROUNDS),77
2,Dustin Johnson,2010,Driving Distance,Driving Distance - (ROUNDS),83
3,Brett Wetterich,2010,Driving Distance,Driving Distance - (ROUNDS),54
4,J.B. Holmes,2010,Driving Distance,Driving Distance - (ROUNDS),100
...,...,...,...,...,...
2740398,Ben Martin,2018,Fairway Bunker Tendency,Fairway Bunker Tendency - (RELATIVE TO PAR),+0.209
2740399,Robert Streb,2018,Fairway Bunker Tendency,Fairway Bunker Tendency - (RELATIVE TO PAR),+0.239
2740400,Andrew Yun,2018,Fairway Bunker Tendency,Fairway Bunker Tendency - (RELATIVE TO PAR),+0.225
2740401,Nate Lashley,2018,Fairway Bunker Tendency,Fairway Bunker Tendency - (RELATIVE TO PAR),+0.143


In [ ]:
pga.printSchema()

root
 |-- Player Name: string (nullable = true)
 |-- Season: string (nullable = true)
 |-- Statistic: string (nullable = true)
 |-- Variable: string (nullable = true)
 |-- Value: integer (nullable = true)



In [ ]:
pga.select('Value').summary('min','max','mean').show()

+-------+------------------+
|summary|             Value|
+-------+------------------+
|    min|              -178|
|    max|           3564954|
|   mean|12494.388998743096|
+-------+------------------+



In [ ]:
pga.coalesce(1).write.mode('overwrite').parquet('pga_parquet.parquet')

In [ ]:
pga.select('Season').distinct().count()

9

In [ ]:
pga.write.mode('overwrite').partitionBy('Season').parquet('pga_parquet_partition.parquet')

In [ ]:
parquet_path = '/content/pga_parquet_partition.parquet'

pga_df = spark.read.parquet(parquet_path)

In [ ]:
pga_df.orderBy('Season').show()

+---------------+----------------+--------------------+-----+------+
|    Player Name|       Statistic|            Variable|Value|Season|
+---------------+----------------+--------------------+-----+------+
|Robert Garrigus|Driving Distance|Driving Distance ...|   71|  2010|
|    Nick Watney|Driving Distance|Driving Distance ...|   91|  2010|
|   Bubba Watson|Driving Distance|Driving Distance ...|   77|  2010|
| Dustin Johnson|Driving Distance|Driving Distance ...|   83|  2010|
|Brett Wetterich|Driving Distance|Driving Distance ...|   54|  2010|
|    J.B. Holmes|Driving Distance|Driving Distance ...|  100|  2010|
|      John Daly|Driving Distance|Driving Distance ...|   63|  2010|
|  Graham DeLaet|Driving Distance|Driving Distance ...|   88|  2010|
|  Angel Cabrera|Driving Distance|Driving Distance ...|   64|  2010|
| Charles Warren|Driving Distance|Driving Distance ...|   64|  2010|
|    D.J. Trahan|Driving Distance|Driving Distance ...|   92|  2010|
|  Martin Flores|Driving Distance|